# Vessel Route Optimization

In [140]:
!pip install ortools

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [141]:
import math
import numpy as np
from numpy import random
from scipy.optimize import fsolve
import ortools
from tabulate import tabulate
from ortools.linear_solver import pywraplp

## Semi-random vectors generation

In [142]:
wind_degrees = []
x_positions = 10
max_change = 0.3
#random.seed(1234)

for day in range(x_positions):
    if day == 0:
        wind_degrees.append(random.randint(0,360))
    else:
        last_day = wind_degrees[day-1]
        wind_degrees.append(random.randint(last_day*(1-max_change),last_day*(1+max_change)))
        
wind_speed = list(random.randint(0,28,(1,x_positions)).flatten())

print("Wind direction (degrees):", wind_degrees)
print("Wind speed (m/s):", wind_speed)

ValueError: high <= 0

## Multiple array problem

In [ ]:
def random_space(max_value, x_positions=5, y_positions=1, day_max_change=0.3, position_max_change=0.2, seed=None):    
    random.seed(seed)
    degrees_matrix = []
    wind_degrees = []
    
    for day in range(x_positions):
        wind_degrees.append(random.randint(1,max_value))
    degrees_matrix.append(wind_degrees) 

    for position in range(0,y_positions-1):
        position_degree = []
        for day in range(x_positions):
            same_day_other_position = abs(degrees_matrix[position-1][day])  
            if same_day_other_position != 0:
                position_degree.append(random.randint(same_day_other_position*(1-position_max_change),same_day_other_position*(1+position_max_change)))
            else: 
                position_degree.append(random.randint(1,max_value))
        degrees_matrix.append(position_degree)
    
    return degrees_matrix

x_positions = 5
y_positions = 3
day_max_change = 0.3
position_max_change = 0.3
seed = 126

wind_direction = random_space(360, x_positions, y_positions, day_max_change, position_max_change)
wind_speed = random_space(27, x_positions, y_positions, day_max_change, position_max_change)

print("Wind direction (degrees):", wind_direction)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [[199, 302, 317, 249, 242], [209, 312, 312, 312, 312], [171, 263, 321, 213, 299]]
Wind speed (m/s): [[10, 13, 18, 8, 20], [12, 11, 14, 8, 22], [11, 10, 18, 8, 19]]


# Open_sail function

In [ ]:
#CONSTANTS TO BE DETERMINED
min_wind_angle = 60 
max_wind_angle = 300 
min_wind_strength = 10

def open_sail(wind_angle, wind_strength):
    open_sail_bool = False 
    
    if wind_angle > min_wind_angle and wind_angle < max_wind_angle and wind_strength > min_wind_strength :
        open_sail_bool = True
        
    #EXCLUDING CRIETRIA FOR THE BOAT_SPEED FUNCTION NOT TO EXPLODE
    if wind_angle > 80 and wind_angle < 120 : open_sail_bool = False
    if wind_angle > 260 and wind_angle < 300 : open_sail_bool = False


    return open_sail_bool


print("We open the sail : ", open_sail(80,40))

We open the sail :  True


# Apparent wind function for boat speed

In [ ]:
# 0<a0<180

def boat_speed(a0,VW,eta):

    f = lambda a: math.sin(math.pi*a0/180) * math.sin(math.pi*a/180) * math.pow((math.sin(math.pi*a/180/2) / math.sin(math.pi*a0/180-math.pi*a/180)), 2)-VW * eta
    a = fsolve(f, [0])[0]
    #print("a:", a)
    VB = VW * (math.sin(math.pi*(a0-a)/180)/math.sin(math.pi*a/180))
    #print("a0:",a0,"and VB:", VB)
    
    return VB

# Problem in 1 dimension

In [ ]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 30 #miles
x_positions = 6 #nb of squares between A and B
eta = 0.001
motor_speed = 22 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle = random_space(360,1,x_positions) #in degrees
wind_speed = random_space(30,1,x_positions) #in mph
square_length = total_distance/x_positions

#defining the lists to be used
list_sail = []
list_boat_speed = []
list_co2 = []
list_time = []

#flatten the list of list into a simple list
wind_angle = [item for sublist in wind_angle for item in sublist]
wind_speed = [item for sublist in wind_speed for item in sublist]

#evaluating if we open the sails for each position
for i in range(x_positions):
    list_sail.append(open_sail(wind_angle[i],wind_speed[i]))
    
#determining the boat speeds depending on whether we use sails on motor   
for i in range(x_positions):
    if list_sail[i]==True:
        list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))
        
    else:
        list_boat_speed.append(motor_speed)
        
#CO2 emissions calculations
for i in range(x_positions):
    if list_sail[i]==True:
        list_co2.append(0)
        
    else:
        list_co2.append(motor_emission_per_mile*square_length)
        
total_emissions = sum(list_co2)
        
#determining the time needed for the journey
for i in range(x_positions):
    list_time.append(square_length/list_boat_speed[i])
        
total_time = sum(list_time)        
        
    
    
#printing results
print("The squares length is", square_length)
print("Wind angles are :", wind_angle)
print("Wind speeds are :", wind_speed)
print("We open the sails :", list_sail)
print("Boat speeds are :", list_boat_speed)
print("The emissions for each square are:", list_co2)
print("The total emissions are:", total_emissions)
print("The time need for each square is:", list_time)
print("The total time needed is:", total_time)

The squares length is 5.0
Wind angles are : [298, 268, 257, 307, 242, 343]
Wind speeds are : [24, 25, 26, 24, 29, 20]
We open the sails : [False, False, True, False, True, False]
Boat speeds are : [22, 22, 55.285843373413464, 22, 58.350706168427685, 22]
The emissions for each square are: [16.0, 16.0, 0, 16.0, 0, 16.0]
The total emissions are: 64.0
The time need for each square is: [0.22727272727272727, 0.22727272727272727, 0.09043906531783978, 0.22727272727272727, 0.08568876588344346, 0.22727272727272727]
The total time needed is: 1.0852187402921922


In [ ]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 30 #miles
x_positions = 6 #nb of squares between A and B
eta = 0.001
motor_speed = 15 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle = random_space(360,1,x_positions, seed =12) #in degrees
wind_speed = random_space(30,1,x_positions, seed =12) #in mph
square_length = total_distance/x_positions

#defining the lists to be used
list_sail = []
list_boat_speed = []
list_co2 = []
list_time = []

#flatten the list of list into a simple list
wind_angle = [item for sublist in wind_angle for item in sublist]
wind_speed = [item for sublist in wind_speed for item in sublist]

for i in range(x_positions):
    #evaluating if we open the sails for each position
    list_sail.append(open_sail(wind_angle[i],wind_speed[i]))
    
    #determining the boat speeds depending on whether we use sails on motor   
    if list_sail[i]==True:
        list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))
        
    else:
        list_boat_speed.append(motor_speed)
        
    #CO2 emissions calculations    
    if list_sail[i]==True:
        list_co2.append(0)
        
    else:
        list_co2.append(motor_emission_per_mile*square_length)
        
    #determining the time needed for the journey    
    list_time.append(square_length/list_boat_speed[i])
        
        
total_emissions = sum(list_co2)     
total_time = sum(list_time)         
    
#printing results
print("The squares length is", square_length)
print("Wind angles are :", wind_angle)
print("Wind speeds are :", wind_speed)
print("We open the sails :", list_sail)
print("Boat speeds are :", list_boat_speed)
print("The emissions for each square are:", list_co2)
print("The total emissions are:", total_emissions)
print("The time need for each square is:", list_time)
print("The total time needed is:", total_time)

The squares length is 5.0
Wind angles are : [332, 395, 268, 392, 262, 335]
Wind speeds are : [12, 12, 10, 11, 11, 11]
We open the sails : [False, False, False, False, False, False]
Boat speeds are : [15, 15, 15, 15, 15, 15]
The emissions for each square are: [16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
The total emissions are: 96.0
The time need for each square is: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
The total time needed is: 1.9999999999999998


# Problem in 2 dimension

In [ ]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 60 #miles
x_positions = 5 #nb of squares between A and B
y_positions = 3
eta = 0.001
motor_speed = 15 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle_matrix = random_space(360, x_positions, y_positions) #in degrees
wind_speed_matrix = random_space(30, x_positions, y_positions) #in mph


def single_line(wind_angle, wind_speed, print_=False):
    
    #defining the lists to be used
    list_sail = []
    list_boat_speed = []
    list_co2 = []
    list_time = []
    
    for i in range(x_positions):
        #evaluating if we open the sails for each position
        list_sail.append(open_sail(wind_angle[i],wind_speed[i]))

        #determining the boat speeds depending on whether we use sails on motor   
        if list_sail[i]==True:
            list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))

        else:
            list_boat_speed.append(motor_speed)

        #CO2 emissions calculations    
        if list_sail[i]==True:
            list_co2.append(0)

        else:
            list_co2.append(motor_emission_per_mile*square_length)

        #determining the time needed for the journey    
        list_time.append(square_length/list_boat_speed[i])    

    #printing results
    if print_:
        print("Wind angles are :", wind_angle)
        print("Wind speeds are :", wind_speed)
        print("We open the sails :", list_sail)
        print("Boat speeds are :", list_boat_speed)
    return (wind_angle,wind_speed, list_sail,list_boat_speed, list_co2)

path = np.zeros((y_positions, x_positions), int)
result_wind_angle = []
result_wind_speed = []
result_sail = []
result_boat_speed = []
result_co2 = []

for y in range(y_positions):
    results = single_line(wind_angle_matrix[y], wind_speed_matrix[y])
    result_wind_angle.append(results[0])
    result_wind_speed.append(results[1])
    result_sail.append(results[2])
    result_boat_speed.append(results[3])
    result_co2.append(results[4])
    
result_co2

[[16.0, 16.0, 16.0, 16.0, 16.0],
 [16.0, 0, 16.0, 16.0, 0],
 [16.0, 16.0, 16.0, 16.0, 0]]

In [ ]:
result_co2[1][2] =16.
result_co2

[[16.0, 16.0, 16.0, 16.0, 16.0],
 [16.0, 0, 16.0, 16.0, 0],
 [16.0, 16.0, 16.0, 16.0, 0]]

In [ ]:
# solver = pywraplp.Solver.CreateSolver('CLP')

# x = {}
# for i in range(y_positions):
#     for j in range(x_positions):
#         x[i, j] = solver.IntVar(0, 1, '')
        
# # Constraints

# # for each x_position, we need the boat to pass by there
# for j in range(x_positions):
#     solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)

# middle_point = y_positions // 2 
# print("middle point: ", middle_point)   
# #choose a specific starting point (i)
# solver.Add(x[middle_point,0] == 1)
# #chose a specific ending point (i)
# #solver.Add(x[middle_point,x_positions-1] == 1)

# # you can only move either the square in front of you either in the two closest diagonals    
# for j in range(x_positions-1):
#     solver.Add(solver.Sum([x[i+1,j+1] + x[i,j+1] + x[i-1, j+1] for i in range(1,y_positions-1)]) <= 1)
#     solver.Add((x[0,j+1] + x[1,j+1]) <= 1)
#     solver.Add((x[y_positions-2,j+1] + x[y_positions-1,j+1]) <= 1)
    
    
#     #for i in range(1,y_positions-1):
#     #    solver.Add(x[i,j+1] == x[i+1,j] + x[i-1, j] + x[i, j])



# solver.Minimize(solver.Sum([result_co2[i][j] * x[i,j] for i in range(y_positions)
#                                                   for j in range(x_positions)]))
# status = solver.Solve()

# print ()

middle point:  1



In [ ]:
## New test

solver = pywraplp.Solver.CreateSolver('CLP')

x = {}
for i in range(y_positions):
    for j in range(x_positions):
        x[i, j] = solver.IntVar(0, 1, '')

# Define the constraints
middle_point = middle_point = y_positions // 2 
# The starting and ending points are fixed
solver.Add(x[middle_point,0] == 1)
solver.Add(x[middle_point,x_positions-1] == 1)


r = [middle_point-1, middle_point, middle_point+1]
print(r)


# For each column, only one variable can take the value 1
for j in range(x_positions):
    solver.Add(solver.Sum([x[i,j] for i in range(y_positions)]) == 1)
    
    


# For each column, the only allowed movements from the previous column are
# to the same row, the up-diagonal, or the low-diagonal
for j in range(1, x_positions):
    for i in range(y_positions-1):
        if i > 0 & i < y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1] + x[i+1,j-1])
        if i == 0:
            solver.Add(x[i,j] <= x[i,j-1] + x[i+1,j-1])
        if i == y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1])
        if j == 1:
            solver.Add(solver.Sum([x[i, j] for i in r]) == 1)



# for each x_position, we need the boat to pass by there
for j in range(x_positions):
    solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)






solver.Minimize(solver.Sum([result_co2[i][j] * x[i,j] for i in range(y_positions)
                                                  for j in range(x_positions)]))
status = solver.Solve()

[0, 1, 2]


In [ ]:
# Print solution.
if status == pywraplp.Solver.OPTIMAL:
    print('Total co2 = ', solver.Objective().Value(), '\n')
    for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('x_position %d took y_position %d.  CO2 = %d' %
                      (j, i, result_co2[i][j]))

Total co2 =  48.0 

x_position 0 took y_position 1.  CO2 = 16
x_position 1 took y_position 1.  CO2 = 0
x_position 2 took y_position 0.  CO2 = 16
x_position 3 took y_position 0.  CO2 = 16
x_position 4 took y_position 1.  CO2 = 0


In [ ]:
for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            path[i,j] = (x[i,j].solution_value())

def show_table(path):
    results = path.tolist()          
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i][j] == 1:
                results[i][j] = "#"
            elif results[i][j] == 0:
                results[i][j] = ""
    print(tabulate(list(map(tuple, results)), tablefmt="grid", stralign="center"))

show_table(path)

+---+---+---+---+---+
|   |   | # | # |   |
+---+---+---+---+---+
| # | # |   |   | # |
+---+---+---+---+---+
|   |   |   |   |   |
+---+---+---+---+---+


## All together just to test test

In [421]:
#ARBITRARY CONSTANTS TO CHECK !!!
total_distance = 60 #miles
x_positions = 7 #nb of squares between A and B
y_positions = 5
eta = 0.001
motor_speed = 15 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle_matrix = random_space(360, x_positions, y_positions) #in degrees
wind_speed_matrix = random_space(30, x_positions, y_positions) #in mph


def single_line(wind_angle, wind_speed, print_=False):
    
    #defining the lists to be used
    list_sail = []
    list_boat_speed = []
    list_co2 = []
    list_time = []
    
    for i in range(x_positions):
        #evaluating if we open the sails for each position
        list_sail.append(open_sail(wind_angle[i],wind_speed[i]))

        #determining the boat speeds depending on whether we use sails on motor   
        if list_sail[i]==True:
            list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))

        else:
            list_boat_speed.append(motor_speed)

        #CO2 emissions calculations    
        if list_sail[i]==True:
            list_co2.append(0)

        else:
            list_co2.append(motor_emission_per_mile*square_length)

        #determining the time needed for the journey    
        list_time.append(square_length/list_boat_speed[i])    

    #printing results
    if print_:
        print("Wind angles are :", wind_angle)
        print("Wind speeds are :", wind_speed)
        print("We open the sails :", list_sail)
        print("Boat speeds are :", list_boat_speed)
    return (wind_angle,wind_speed, list_sail,list_boat_speed, list_co2)

path = np.zeros((y_positions, x_positions), int)
result_wind_angle = []
result_wind_speed = []
result_sail = []
result_boat_speed = []
result_co2 = []

for y in range(y_positions):
    results = single_line(wind_angle_matrix[y], wind_speed_matrix[y])
    result_wind_angle.append(results[0])
    result_wind_speed.append(results[1])
    result_sail.append(results[2])
    result_boat_speed.append(results[3])
    result_co2.append(results[4])
    
result_co2


result_co2[1][2] =16.
result_co2


## New test

solver = pywraplp.Solver.CreateSolver('CLP')

x = {}
for i in range(y_positions):
    for j in range(x_positions):
        x[i, j] = solver.IntVar(0, 1, '')

# Define the constraints
middle_point = middle_point = y_positions // 2 
# The starting and ending points are fixed
solver.Add(x[middle_point,0] == 1)
solver.Add(x[middle_point,x_positions-1] == 1)


r = [middle_point-1, middle_point, middle_point+1]
print(r)


# For each column, only one variable can take the value 1
for j in range(x_positions):
    solver.Add(solver.Sum([x[i,j] for i in range(y_positions)]) == 1)
    # Force second column constraint to follow from middle point and be have only allowed potential solutions
    


# For each column, the only allowed movements from the previous column are
# to the same row, the up-diagonal, or the low-diagonal
for j in range(1, x_positions):
    for i in range(y_positions-1):
        if i > 0 & i < y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1] + x[i+1,j-1])
        if i == 0:
            solver.Add(x[i,j] <= x[i,j-1] + x[i+1,j-1])
        if i == y_positions-1:
            solver.Add(x[i,j] <= x[i,j-1] + x[i-1,j-1])
        if j == 1:
            solver.Add(solver.Sum([x[i, j] for i in r]) == 1)


# for each x_position, we need the boat to pass by there
for j in range(x_positions):
    solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)





solver.Minimize(solver.Sum([result_co2[i][j] * x[i,j] for i in range(y_positions)
                                                  for j in range(x_positions)]))
status = solver.Solve()



# Print solution.
if status == pywraplp.Solver.OPTIMAL:
    print('Total co2 = ', solver.Objective().Value(), '\n')
    for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('x_position %d took y_position %d.  CO2 = %d' %
                      (j, i, result_co2[i][j]))


for j in range(x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            path[i,j] = (x[i,j].solution_value())

def show_table(path):
    results = path.tolist()          
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i][j] == 1:
                results[i][j] = "#"
            elif results[i][j] == 0:
                results[i][j] = ""
    print(tabulate(list(map(tuple, results)), tablefmt="grid", stralign="center"))

show_table(path)

[1, 2, 3]
Total co2 =  32.0 

x_position 0 took y_position 2.  CO2 = 0
x_position 1 took y_position 1.  CO2 = 0
x_position 2 took y_position 0.  CO2 = 16
x_position 3 took y_position 0.  CO2 = 0
x_position 4 took y_position 4.  CO2 = 16
x_position 5 took y_position 3.  CO2 = 0
x_position 6 took y_position 2.  CO2 = 0
+---+---+---+---+---+---+---+
|   |   | # | # |   |   |   |
+---+---+---+---+---+---+---+
|   | # |   |   |   |   |   |
+---+---+---+---+---+---+---+
| # |   |   |   |   |   | # |
+---+---+---+---+---+---+---+
|   |   |   |   |   | # |   |
+---+---+---+---+---+---+---+
|   |   |   |   | # |   |   |
+---+---+---+---+---+---+---+


# Creating measures of the field to calculate distance and velocity

In [12]:
tot_y_distance = 18 #miles
tot_x_distance = total_distance #miles
y_square = tot_y_distance/y_positions
x_square = tot_x_distance/x_positions

In [13]:
coordinates_field = [[(y_square*i,x_square*j) for j in range(x_positions)] for i in range(y_positions)]
print(tabulate(coordinates_field, tablefmt="grid", stralign="center"))

+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+
| (0.0, 0.0)  | (0.0, 7.5)  | (0.0, 15.0)  | (0.0, 22.5)  | (0.0, 30.0)  | (0.0, 37.5)  | (0.0, 45.0)  | (0.0, 52.5)  |
+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+
| (6.0, 0.0)  | (6.0, 7.5)  | (6.0, 15.0)  | (6.0, 22.5)  | (6.0, 30.0)  | (6.0, 37.5)  | (6.0, 45.0)  | (6.0, 52.5)  |
+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+
| (12.0, 0.0) | (12.0, 7.5) | (12.0, 15.0) | (12.0, 22.5) | (12.0, 30.0) | (12.0, 37.5) | (12.0, 45.0) | (12.0, 52.5) |
+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+


In [14]:
# we start from (10,0)
start_point = coordinates_field[1][0]
# we go straight
straight_point = coordinates_field[1][1]
# we go diagonally
diagonal_point = coordinates_field[0][1]

def get_distance(starting_point, end_point):
    return math.sqrt((end_point[0]-starting_point[0])**2 + (end_point[1]-starting_point[1])**2)

get_distance(start_point, diagonal_point)

9.604686356149273

In [15]:
def get_path_distance(path, coordinates_field):
    passed_by = []
    for j in range(x_positions):
        for i in range(y_positions):
            if path[i][j] == 1:
                passed_by.append(coordinates_field[i][j])

    total_distances = []            
    for way in range(len(passed_by)-1):
        total_distances.append(get_distance(passed_by[way], passed_by[way+1]))
    return total_distances

get_path_distance(path, coordinates_field)

[7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5]

In [16]:
path

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0]])

# Experiment

In [18]:
#ARBITRARY CONSTANTS TO CHECK !!!
tot_y_distance = 60 #miles
tot_x_distance = total_distance
x_positions = 8 #nb of squares between A and B
y_positions = 3
eta = 0.001
motor_speed = 15 #in mph
motor_emission_per_mile = 3.2 #in kgCO2 per mile

wind_angle_matrix = random_space(360, x_positions, y_positions, seed =12) #in degrees
wind_speed_matrix = random_space(30, x_positions, y_positions, seed =12) #in mph


def single_line(wind_angle, wind_speed, print_=False):
    
    #defining the lists to be used
    list_sail = []
    list_boat_speed = []
    list_co2 = []
    list_time = []
    
    for i in range(x_positions):
        #evaluating if we open the sails for each position
        list_sail.append(open_sail(wind_angle[i],wind_speed[i]))

        #determining the boat speeds depending on whether we use sails on motor   
        if list_sail[i]==True:
            list_boat_speed.append(boat_speed(wind_angle[i],wind_speed[i],eta))

        else:
            list_boat_speed.append(motor_speed)

        #CO2 emissions calculations    
        if list_sail[i]==True:
            list_co2.append(0)

        else:
            list_co2.append(motor_emission_per_mile*square_length)

        #determining the time needed for the journey    
        list_time.append(square_length/list_boat_speed[i])    

    #printing results
    if print_:
        print("Wind angles are :", wind_angle)
        print("Wind speeds are :", wind_speed)
        print("We open the sails :", list_sail)
        print("Boat speeds are :", list_boat_speed)
    return (wind_angle, wind_speed, list_sail,list_boat_speed, list_co2)

path = np.zeros((y_positions, x_positions), int)
result_wind_angle = []
result_wind_speed = []
result_sail = []
result_boat_speed = []
result_co2 = []

for y in range(y_positions):
    results = single_line(wind_angle_matrix[y], wind_speed_matrix[y])
    result_wind_angle.append(results[0])
    result_wind_speed.append(results[1])
    result_sail.append(results[2])
    result_boat_speed.append(results[3])
    result_co2.append(results[4])
    
get_sail_integer = np.vectorize(int)
sail_integer = get_sail_integer(result_sail)

get_motor = np.vectorize(np.logical_not)
motor_integer = get_sail_integer(get_motor(result_sail))

In [19]:
sail_integer

array([[0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0]])

In [20]:
motor_integer

array([[1, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 1, 1]])

In [21]:
fixed = {"speed":result_boat_speed, "sail":sail_integer, "motor":motor_integer}
tot_y_distance = 60 #miles
tot_x_distance = total_distance #miles
y_square = tot_y_distance/y_positions
x_square = tot_x_distance/x_positions
coordinates_field = [[(y_square*i,x_square*j) for j in range(x_positions)] for i in range(y_positions)]
# print(tabulate(coordinates_field, tablefmt="grid", stralign="center"))


In [84]:
solver = pywraplp.Solver.CreateSolver('CLP')

x = {}
for i in range(y_positions):
    for j in range(x_positions):
        x[i, j] = solver.IntVar(0, 1, '')
        
# Constraints

# for each x_position, we need the boat to pass by there
for j in range(x_positions):
    solver.Add(solver.Sum([x[i, j] for i in range(y_positions)]) == 1)

# you can only move either the square in front of you either in the two closest diagonals    
for j in range(1, x_positions):
    solver.Add(solver.Sum([x[i+1,j-1] + x[i,j-1] + x[i-1, j-1] for i in range(1,y_positions-1)]) == 1)
    solver.Add((x[0,j-1] + x[1,j-1]) == 1)
    solver.Add((x[y_positions-2,j-1] + x[y_positions-1,j-1]) == 1)

middle_point = y_positions // 2    
#choose a specific starting point (i)
solver.Add(x[middle_point,0] == 1)
#chose a specific ending point (i)
solver.Add(x[0,x_positions-1] == 1)


def get_distance(starting_point, end_point):
    return math.sqrt((end_point[0]-starting_point[0])**2 + (end_point[1]-starting_point[1])**2)

solver.Minimize(solver.Sum([x[i,j] * motor_integer[i][j] * motor_emission_per_mile for i in range(y_positions)
                                                  for j in range(1,x_positions)]))
status = solver.Solve()

In [87]:
# Print solution.
if status == pywraplp.Solver.OPTIMAL:
    print('Total co2 = ', solver.Objective().Value(), '\n')
    for j in range(1,x_positions):
        for i in range(y_positions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('x_position %d took y_position %d.  CO2 = %d' %
                      (j, i, 999))

Total co2 =  16.0 

x_position 1 took y_position 1.  CO2 = 999
x_position 2 took y_position 1.  CO2 = 999
x_position 3 took y_position 1.  CO2 = 999
x_position 4 took y_position 1.  CO2 = 999
x_position 5 took y_position 1.  CO2 = 999
x_position 6 took y_position 1.  CO2 = 999
x_position 7 took y_position 0.  CO2 = 999


In [88]:
path

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [37]:
print(coordinates_field[0][1])
print(coordinates_fiel[])

(0.0, 7.5)

In [71]:
coordinates_field[1][2]

(20.0, 15.0)

In [95]:
passed_by = list(map(list,path))
for j in range(x_positions):
    for i in range(y_positions):
        passed_by[i][j] = passed_by[i][j] * get_distance(coordinates_field[i][j], coordinates_field[i][j-1])

passed_by
#def get_distance(starting_point, end_point):

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [52.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [93]:
def get_path_distance(path, coordinates_field):
    passed_by = list(map(list,path))
    for j in range(1,x_positions+1):
        for i in range(y_positions):
            passed_by[i][j] = passed_by[i][j] * get_distance(coordinates_field[i][j], coordinates_field[i][j-1])
     
    result = path
    for i in range(y_positions):
        for j in range(x_positions):
            result[i,j] += get_distance(passed_by[way], passed_by[way+1])
    return result

get_path_distance(path, coordinates_field)

IndexError: list index out of range

In [90]:
path

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [50]:
passed_by = np.zeros((y_positions, x_positions), int)
passed_by[i,j] = path[i,j] + passed_by[i,j]
get_distance(passed_by[1], passed_by[1+1])

0.0

In [48]:
def get_distance(starting_point, end_point):
    return math.sqrt((end_point[0]-starting_point[0])**2 + (end_point[1]-starting_point[1])**2)

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])